In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import utils
from torch.utils.data import DataLoader
import tqdm as tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from utils import load_csv, drop_cols, remove_strings, groupedAvg, subsample, normalize_df
from create_dataset import AnimalDataset, AnimalDatasetEmbedding, UnpairedDataset
from generators import EmbeddingEncoderDecoderGenerator, EmbeddingUnetGenerator, OneHotGenerator
from discriminators import MultiChannelDiscriminator 

In [3]:
# config

ALLDATA = True # if False -> smaller dataset
SKIPCONNECTIONS = True
EMBEDDING = True
ONEHOTENCODING = True
UNPAIRED = False

# DISCRIMINATOR = MultiChannelDiscriminator
# GENERATOR = EmbeddingUnetGenerator

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
LEARNING_RATE = 1e-2  # 1e-5 was too small for 'LVtot_kalibriert' and 'LVtot' 
NUM_WORKERS = 16
NUM_EPOCHS = 30
LR_DECAY_AFTER_EPOCH = 200  
GENERATION_AFTER_EPOCH = NUM_EPOCHS # number of epochs after which the model generates a sample
SIG_A = "AoP"           # Drucksignal Hauptschlagader = Aortendruck
SIG_B = "VADcurrent"    # VAD Strom [A] – Pumpemstrom in Ampere
SIG_C = "VadQ"          # Fluss durch VAD (VAD = Ventrikular assistance device = Pumpe) = Pumpenfluss
SIG_D = "LVP"           # Ventrikeldruck links = Drucksignal der linken Herzkammer
TARGET = "LVtot_kalibriert"
source_signals = [SIG_A, SIG_C, SIG_D]
CHANNELS = len(source_signals)
WINDOW = 256
# target = TARGET

# Use adversarial loss
GAN_LOSS = True   # adversarial loss
LAMBDA_GAN = 1.0
# Use cycle consistency loss
CYCLE = True
LAMBDA_CYCLE = 1.0
# Use supervised loss
SUPERVISED = False 
LAMBDA_SUPERVISED = 1.0
# Use Identity loss
IDENTITY = False
LAMBDA_IDENTITY = 10.0

In [ ]:
# def load_df(path):
#     df = utils.load_csv(path)
#     df = utils.drop_cols(df)
#     df = df.dropna()
#     return df

# def load_data(alldata=ALLDATA):
#     if not alldata:
#         path = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/LeRntVAD_csv_exports_not_all_the_data/constant_speed_interventions/" 
#         df = load_df(path)
#         print(df.shape)
#         df = utils.remove_strings(df)
#         df = utils.subsample(df, 10)
#         print(df.shape)
#         return df

#     else:
#         path_1 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_1"  
#         path_2 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_2"
#         # path_3 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_3"
#         # path_4 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_4"
        
#         df_1 = load_csv(path_1)
#         df_2 = load_csv(path_2)
#         # df_3 = load_csv(path_3)
#         # df_4 = load_csv(path_4)
#         df = pd.concat([df_1, df_2], axis=0, ignore_index=True)

#         print(df.shape)
#         df = utils.remove_strings(df)
#         df = utils.subsample(df, 10)
#         print(df.shape)
#         return df

# df = load_data(alldata=ALLDATA)

In [4]:
# if ALLDATA == False:
#     # load only a small part of the data and drop the unnecessary columns
#     # paths of only a small part of the data
#     path = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/LeRntVAD_csv_exports_not_all_the_data/constant_speed_interventions/" 
#     df = utils.load_csv(path)
#     df = utils.drop_cols(df)
#     df = df.dropna()

#     # select only rows where 'Phasenzuordnung' is 1
#     # df = df.loc[df['Phasenzuordnung'] == 1]

#     print(df.shape) 

(8072873, 13)


### Load data and preprocess

In [ ]:
if ALLDATA == True:
    # Load all the data and drop unnecessary columns
    # We load the data separately, to avoid a Runtime error

    # all the data
    path_1 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_1"  
    path_2 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_2"
    path_3 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_3"
    path_4 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_4"

    df_1 = utils.load_csv(path_1)
    df_1 = utils.drop_cols(df_1)

    df_2 = utils.load_csv(path_2)
    df_2 = utils.drop_cols(df_2)

    df_3 = utils.load_csv(path_3)
    df_3 = utils.drop_cols(df_3)

    df_4 = utils.load_csv(path_4)
    df_4 = utils.drop_cols(df_4)

    # concatenate the separate dataframes
    df = pd.concat([df_1, df_2, df_3, df_4], axis=0, ignore_index=True)
    df = df.dropna()

    print('Size of the whole dataset',df.shape)
    df = utils.remove_strings(df)
    df = utils.subsample(df, 10)
    print('Size of the dataset after subsampling',df.shape)

In [ ]:
# import os


# def load_df(path):
#     df = utils.load_csv(path)
#     df = utils.drop_cols(df)
#     df = df.dropna()
#     return df


# if ALLDATA == True:
#     # Load all the data and drop unnecessary columns
#     # We load the data separately, to avoid a Runtime error

#     # all the data
#     base_path = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/"
#     paths = [os.path.join(base_path, f"Data_split_{i}") for i in range(1, 5)]

#     df = pd.concat([load_df(path) for path in paths], axis=0, ignore_index=True)
    
#     print('Size of the whole dataset',df.shape)
#     # select only rows where 'Phasenzuordnung' is 1
#     # df = df.loc[df['Phasenzuordnung'] == 1]
#     # print('Size of dataset with only the first phase',df.shape)
# else:
#     # load only a small part of the data and drop the unnecessary columns
#     # paths of only a small part of the data
#     path = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/LeRntVAD_csv_exports_not_all_the_data/constant_speed_interventions/" 

#     df = load_df(path)

#     # select only rows where 'Phasenzuordnung' is 1
#     # df = df.loc[df['Phasenzuordnung'] == 1]

#     print(df.shape) 

In [ ]:
print(len(df['animal'].unique()))
# remove animals with less than 10 data points
df = df.groupby('animal').filter(lambda x: len(x) > 10)
print('Number o fanimals after removing those with less than 10 data points', df['animal'].unique())

In [ ]:
# def normalize_by_phase1(df, scaler):
#     '''
#     Normalize the data by the first phase
#     '''
#     # column names
#     cols = df.columns.tolist()
#     # select data von Phasenzuordnung == 1
#     phase_1 = df.loc[df['Phasenzuordnung'] == 1]
#     phase_1 = phase_1.to_numpy() 
#     df = df.to_numpy()  #convert to numpy
#     # scale the data only with the first phase
#     scaler.fit(phase_1)
#     transformed_data = scaler.transform(df)
#     df = pd.DataFrame(transformed_data, columns=cols)  # convert to dataframe
#     # df = df.join(df_IPA) 
#     return df

In [ ]:


# def normalize_df(df, scaler):
#     df_IPA = df[['intervention', 'Phasenzuordnung', 'animal']]
#     df_temp = pd.DataFrame()
#     scaler = StandardScaler()

#     for animal in df['animal'].unique():
#         # split df into separate dataframes for each animal
#         df_animal = df.loc[df['animal'] == animal]
#         df_animal = utils.normalize_by_phase1(df_animal, scaler) # normalize by phase 1
#         # append df_animal to df_temp
#         df_temp = pd.concat([df_temp, df_animal], axis=0, ignore_index=True)

#     df = df_temp
#     df = df.drop(columns=['intervention', 'Phasenzuordnung', 'animal'])
#     df.dropna(inplace=True)
#     df = df.join(df_IPA)
#     return df

df = utils.normalize_df(df)

In [ ]:
# df_IPA = df[['intervention', 'Phasenzuordnung', 'animal']]

# scaler = StandardScaler()
# df_temp = pd.DataFrame()

# for animal in df['animal'].unique():
#     # split df into separate dataframes for each animal
#     df_animal = df.loc[df['animal'] == animal]
#     df_animal = utils.normalize_by_phase1(df_animal, scaler) # normalize by phase 1
#     # append df_animal to df_temp
#     df_temp = pd.concat([df_temp, df_animal], axis=0, ignore_index=True)

# df = df_temp
# df = df.drop(columns=['intervention', 'Phasenzuordnung', 'animal'])
# df.dropna(inplace=True)
# df = df.join(df_IPA)
utils.visualize(df, [SIG_A, SIG_B, SIG_C, SIG_D, 'intervention', 'Phasenzuordnung', 'animal'], 5000)

In [ ]:
# group by animal size in percent
df_grouped = df.groupby('animal').size().reset_index(name='counts')
df_grouped['counts'] = df_grouped['counts'] / df_grouped['counts'].sum() * 100
# df_grouped

#### Pick always the same test animals

In [ ]:
# pick 1 random animals for test data
test_animals = df['animal'].sample(n=1, random_state=42).unique()

all_animals = df['animal'].unique()
# remove test animals from train animals
train_animals =  [x for x in all_animals if x not in test_animals]

print('\nTest animal(s):', test_animals)
# test data
df_test = df[df['animal'].isin(test_animals)]

# train dataframe with only animals from train_animals
df_train = df[df['animal'].isin(train_animals)]
print('\nDifferent animal IDs after removing those that are in the test dataset: ',len(df_train['animal'].unique()))


print('\nTrain data shape:', df_train.shape)
print('\nTest data shape:', df_test.shape)

# lengt of df_train
print('\nThe test dataset is {} percent of the whole data: '.format((len(df_test)/(len(df_train) + len(df_test))) * 100))

In [ ]:
# def double_conv_pad(in_channels, out_channels):
#     return nn.Sequential(
#         nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, padding_mode='zeros'),
#         nn.InstanceNorm1d(out_channels),
#         nn.LeakyReLU(inplace=True),
#         nn.Dropout1d(p=0.1, inplace=False),
#         nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1, padding_mode='zeros'),
#         nn.InstanceNorm1d(out_channels),
#         nn.LeakyReLU(inplace=True),
#         nn.Dropout1d(p=0.1, inplace=False),
#     )

# class OneHotGenerator(nn.Module):
#     def __init__(self, INPUTCHANNELS, OUTPUTCHANNELS):
#         super(OneHotGenerator, self).__init__()
#         self.maxpool = nn.MaxPool1d((2))  

#         self.down_conv1 = double_conv_pad(INPUTCHANNELS, 32) 
#         self.down_conv2 = double_conv_pad(32, 64) 
#         self.down_conv3 = double_conv_pad(64, 128)
#         self.down_conv4 = double_conv_pad(128, 256)
#         self.down_conv5 = double_conv_pad(256, 512)
#         self.down_conv6 = double_conv_pad(512, 1024)
        
#         self.phaseLinear = nn.Linear(6, 32)
#         self.interventionLinear = nn.Linear(11, 32)
#         self.FCLinear = nn.Linear(96, 32)

#         self.up_trans1 = nn.ConvTranspose1d(1024, 512, kernel_size=(2), stride=2, padding=0)
#         self.up_conv1 = double_conv_pad(1024, 512)
#         self.up_trans2 = nn.ConvTranspose1d(512, 256, kernel_size=(2), stride=2, padding=0)
#         self.up_conv2 = double_conv_pad(512, 256)
#         self.up_trans3 = nn.ConvTranspose1d(256, 128, kernel_size=(2), stride=2, padding=0)
#         self.up_conv3 = double_conv_pad(256, 128)
#         self.up_trans4 = nn.ConvTranspose1d(128, 64, kernel_size=(2), stride=2, padding=0)
#         self.up_conv4 = double_conv_pad(128, 64)
#         self.up_trans5 = nn.ConvTranspose1d(64, 32, kernel_size=(2), stride=2, padding=0)
#         self.up_conv5 = double_conv_pad(64, 32)

#         self.out = nn.Conv1d(32, OUTPUTCHANNELS, kernel_size=1) # kernel_size must be == 1

#         self.apply(self._init_weights)
        
#     def _init_weights(self, module):
#         if isinstance(module, nn.Conv1d):
#             module.weight.data.normal_(mean=0.0, std=1)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.ConvTranspose1d):
#             module.weight.data.normal_(mean=0.0, std=1)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=1)
#         elif isinstance(module, nn.BatchNorm1d):
#             module.weight.data.normal_(mean=0.0, std=1)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=1)
#             if module.bias is not None:
#                 module.bias.data.zero_()


#     def forward(self, input, phase, intervention):
#         # [Batch size, Channels in, Height, Width]
        
#         # downsampling
#         x1 = self.down_conv1(input)   
#         x2 = self.maxpool(x1) 
#         x3 = self.down_conv2(x2)  
#         x4 = self.maxpool(x3) 
#         x5 = self.down_conv3(x4) 
#         x6 = self.maxpool(x5) 
#         x7 = self.down_conv4(x6)
#         x8 = self.maxpool(x7)
#         x9 = self.down_conv5(x8)
#         x10 = self.maxpool(x9)
#         x11 = self.down_conv6(x10)     

#         # one hot encoding
#         p = F.one_hot(phase, num_classes=6).type(torch.FloatTensor).to(DEVICE)
#         i = F.one_hot(intervention, num_classes=11).type(torch.FloatTensor).to(DEVICE)
#         # Fully connected Layers
#         p = self.phaseLinear(p).to(DEVICE)
#         i = self.interventionLinear(i).to(DEVICE)
#         # Concatenate the phase and intervention one hot encodings with the output of the last convolutional layer and reshape
#         x7 = torch.cat([x7, p, i], 1) 
#         x7 = x7.reshape(x7.shape[0], WINDOW, 96)
#         x7 = self.FCLinear(x7).to(DEVICE) 
        

#         x = self.up_trans1(x11)
#         x = self.up_conv1(torch.cat([x, x9], 1))  # skip connection
#         x = self.up_trans2(x)
#         x = self.up_conv2(torch.cat([x, x7], 1))  # skip connection
#         x = self.up_trans3(x)
#         x = self.up_conv3(torch.cat([x, x5], 1))  # skip connection
#         x = self.up_trans4(x)
#         x = self.up_conv4(torch.cat([x, x3], 1))  # skip connection
#         x = self.up_trans5(x)
#         x = self.up_conv5(torch.cat([x, x1], 1))  # skip connection
#         x = self.out(x)
#         return x

In [ ]:
if SKIPCONNECTIONS and not ONEHOTENCODING: # we always use embeddings
    gen_B = EmbeddingUnetGenerator(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1).to(DEVICE)
    gen_A = EmbeddingUnetGenerator(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS).to(DEVICE)

if not SKIPCONNECTIONS and not ONEHOTENCODING:
    gen_B = EmbeddingEncoderDecoderGenerator(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1).to(DEVICE)
    gen_A = EmbeddingEncoderDecoderGenerator(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS).to(DEVICE)

if SKIPCONNECTIONS and ONEHOTENCODING:
    gen_B = OneHotGenerator(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1).to(DEVICE)
    gen_A = OneHotGenerator(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS).to(DEVICE)

disc_B = MultiChannelDiscriminator(CHANNELS = 1).to(DEVICE)
disc_A = MultiChannelDiscriminator(CHANNELS = CHANNELS).to(DEVICE)

# optimizers for discriminator and generator 
opt_disc = torch.optim.AdamW(                                         
    list(disc_A.parameters()) + list(disc_B.parameters()), 
    lr=LEARNING_RATE, 
)
opt_gen = torch.optim.AdamW(
    list(gen_A.parameters()) + list(gen_B.parameters()),
    lr=LEARNING_RATE,
)

# maybe a step learning rate would be a good idea 1e-4 -> 1e-5
# scheduler
# gen_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer = opt_gen, milestones=[10, 30, 50, 80], gamma=0.1)
                                                    
# disc_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer = opt_disc, milestones=[10, 30, 50, 80], gamma=0.1)

gen_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = opt_gen,
                                                           factor=0.1, patience=3, threshold=1e-4,
                                                           min_lr=1e-6,
                                                    )
disc_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = opt_disc,
                                                            factor=0.1, patience=3, threshold=1e-4,
                                                            min_lr=1e-6,
                                                    )


# losses
l1 = nn.L1Loss() 
mse = nn.MSELoss() 

if UNPAIRED:
    train_dataset = UnpairedDataset(df_train, source_signals, target_name = TARGET, test = False, window_length = WINDOW)
    test_dataset = UnpairedDataset(df_test, source_signals, target_name = TARGET, test = True, window_length = WINDOW)
    gen_dataset = UnpairedDataset(df_test, source_signals, target_name = TARGET, test = False, window_length = WINDOW)


if not UNPAIRED:
    # create datasets
    train_dataset = AnimalDatasetEmbedding(df_train, source_signals, target_name = TARGET, test = False, window_length = WINDOW)
    test_dataset = AnimalDatasetEmbedding(df_test, source_signals, target_name = TARGET, test = True, window_length = WINDOW)
    gen_dataset = AnimalDatasetEmbedding(df_test, source_signals, target_name = TARGET, test = False, window_length = WINDOW)


# Data loader
loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True,)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True,)
gen_loader = DataLoader(gen_dataset, batch_size=1, shuffle=True, pin_memory=True,)

# run in float16
# g_scaler = torch.cuda.amp.GradScaler()
# d_scaler = torch.cuda.amp.GradScaler()

In [ ]:
import wandb

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Cycle_GAN",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LEARNING_RATE,
    "epochs": NUM_EPOCHS,
    }
)

In [ ]:
def gen_signals(fake_target, fake_source, target, source):
    fake_target = fake_target.reshape(-1)
    fake_source = fake_source.reshape(-1)
    source = source.reshape(-1)
    target = target.reshape(-1)

    fig, ax = plt.subplots(2, 1, figsize=(10, 8))
    ax[0].plot(source.cpu().detach().numpy(), label= 'Real source signals')
    ax[0].plot(fake_source.cpu().detach().numpy(), label= 'Recreated source signals')
    ax[0].set_xlabel('Signal length')
    ax[0].set_ylabel('Loss')
    ax[0].legend()

    ax[1].plot(target.cpu().detach().numpy(), label= 'Real target signal')
    ax[1].plot(fake_target.cpu().detach().numpy(), label= 'Recreated target signal')
    ax[1].set_xlabel('Signal length')
    ax[1].set_ylabel('Loss')
    ax[1].legend()

def discriminator_loss(disc, reals, fakes):
    # calculate how close reals are to being classified as real
    real_loss = mse(disc(reals), torch.ones_like(disc(reals)))
    # calculate how close fakes are to being classified as fake
    fake_loss = mse(disc(fakes), torch.zeros_like(disc(fakes)))
    # return the average of real and fake loss
    return (real_loss + fake_loss) / 2


# @torch.cuda.amp.autocast()
def get_disc_loss(sig_A, sig_B, phase, intervention, 
                    disc_A, disc_B, gen_A, gen_B, fake_A, fake_B
                    ):
    """
    Return the loss of the discriminator given inputs.
    """
    # generate fakes
    # with torch.no_grad():
    #     fake_B = gen_B(sig_A, phase, intervention).detach()
    #     fake_A = gen_A(sig_B, phase, intervention).detach()
    
    # discriminator loss
    disc_B_loss = discriminator_loss(disc_B, sig_B, fake_B)
    disc_A_loss = discriminator_loss(disc_A, sig_A, fake_A)
    disc_loss = (disc_A_loss + disc_B_loss) / 2

    return disc_loss, disc_A_loss, disc_B_loss

# @torch.cuda.amp.autocast()
def calc_gen_loss(sig_A, sig_B, phase, intervention,
                  gen_A, gen_B, disc_A, disc_B, fake_B, fake_A
                  ):
    loss = 0

    if GAN_LOSS:
        g_A_loss = mse(disc_A(fake_A), torch.ones_like(disc_A(fake_A))) 
        g_B_loss = mse(disc_B(fake_B), torch.ones_like(disc_B(fake_B))) 

        loss += g_A_loss * LAMBDA_GAN + g_B_loss * LAMBDA_GAN
    else:
        g_A_loss = torch.tensor(0)
        g_B_loss = torch.tensor(0)

    if CYCLE:
        rec_B = gen_B(fake_A, phase, intervention)
        rec_A = gen_A(fake_B, phase, intervention)
        cycle_B_loss = l1(sig_B, rec_B)  # l1 loss: Mean absolute error between each element in the input x and target y
        cycle_A_loss = l1(sig_A, rec_A)  # l1 loss in cycle GAN paper

        loss += cycle_B_loss * LAMBDA_CYCLE + cycle_A_loss * LAMBDA_CYCLE
    else:
        cycle_B_loss = torch.tensor(0)
        cycle_A_loss = torch.tensor(0)

    if SUPERVISED:
        sup_A_loss = mse(sig_A, fake_A)
        sup_B_loss = mse(sig_B, fake_B)

        loss += sup_A_loss * LAMBDA_SUPERVISED + sup_B_loss * LAMBDA_SUPERVISED
    else:
        sup_A_loss = torch.tensor(0)
        sup_B_loss = torch.tensor(0)

    if IDENTITY:
        id_B_loss = l1(sig_B, gen_B(sig_B, phase, intervention))
        id_A_loss = l1(sig_A, gen_A(sig_A, phase, intervention))

        loss += id_B_loss * LAMBDA_IDENTITY + id_A_loss * LAMBDA_IDENTITY
    else:
        id_B_loss = torch.tensor(0)
        id_A_loss = torch.tensor(0)

    return loss, g_A_loss, g_B_loss, cycle_B_loss, cycle_A_loss, id_B_loss, id_A_loss, sup_A_loss, sup_B_loss

## Training

In [ ]:
# training loop
from tqdm.auto import tqdm

for epoch in tqdm(range(NUM_EPOCHS)):

    for source, target, phase, intervention in loader:
        # convert to float16
        source = source.float() # neccessary to prevent error: "Input type (torch.cuda.DoubleTensor) 
        target = target.float() # and weight type (torch.cuda.HalfTensor) should be the same"
    
        # move to GPU
        source = source.to(DEVICE)
        target = target.to(DEVICE)
        phase = phase.to(DEVICE)
        intervention = intervention.to(DEVICE)

        #  ------------------------------- #
        #  ----- train discriminators ---- #
        #  ------------------------------- #
        with torch.no_grad():
            fake_target = gen_B(source, phase, intervention).detach()
            fake_source = gen_A(target, phase, intervention).detach()

        d_loss, disc_A_loss, disc_B_loss = get_disc_loss(source, target, phase, intervention,
                                                            disc_A, disc_B, gen_A, gen_B, 
                                                            fake_source, fake_target
                                                        )

        # update gradients of discriminator 
        opt_disc.zero_grad() 
        d_loss.backward()
        opt_disc.step()
        # d_scaler.scale(d_loss).backward()  
               

        # -------------------------------- #
        # ------- train generators ------- #
        # -------------------------------- # 

        out = calc_gen_loss(source, target, phase, intervention,
                                gen_A, gen_B, disc_A, disc_B, 
                                fake_target, fake_source
                                )
        g_loss, g_A_loss, g_B_loss, cycle_B_loss, cycle_A_loss, id_B_loss, id_A_loss, sup_A_loss, sup_B_loss = out

        # update gradients of generator
        opt_gen.zero_grad()
        g_loss.backward()
        opt_gen.step()
        # g_scaler.scale(g_loss).backward()

    # Optimizer step
    # d_scaler.step(opt_disc)  
    # d_scaler.update()

    # g_scaler.step(opt_gen) 
    # g_scaler.update()

    wandb.log({'Train/Discriminator A loss': disc_A_loss.item(),
                'Train/Discriminator B loss': disc_B_loss.item(),
                'Train/Total Discriminator loss': d_loss.item(),
                'Train/Total Generator loss': g_loss.item(),
                'Train/Adversarial loss A': g_A_loss.item(),
                'Train/Adversarial loss B': g_B_loss.item(),
                'Train/Cycle consistency loss A': cycle_A_loss.item(),
                'Train/Cycle consistency loss B': cycle_B_loss.item(),
                'Train/Supervised loss A': sup_A_loss.item(),
                'Train/Supervised loss B': sup_B_loss.item(),
                'Learning rate': opt_gen.param_groups[0]["lr"],
                # 'Train/Identity loss A': id_A_loss.item(),
                # 'Train/Identity loss B': id_B_loss.item()
                })
        

    # ------------------------ #
    # ------ Validation ------ #
    # ------------------------ #

    if (epoch+1) % 1 == 0:

        with torch.no_grad():
            # set models to evaluation mode
            disc_A.eval()  # set discriminator to evaluation mode
            disc_B.eval()  # turns off Dropouts Layers, BatchNorm Layers etc
            gen_B.eval()
            gen_A.eval()

            for source, target, phase, intervention in test_loader:
                # convert to float16
                source = source.float()
                target = target.float()

                # move to GPU
                source = source.to(DEVICE)
                target = target.to(DEVICE)
                phase = phase.to(DEVICE)
                intervention = intervention.to(DEVICE)

                fake_target = gen_B(source, phase, intervention).detach() # already torch.no_grad()
                fake_source = gen_A(target, phase, intervention).detach()

                # generate signals during validation
                #gen_signals(fake_target, fake_source, target, source)

                # calculate l1 loss of fake signals and real signals
                test_real_fake_lossB= l1(target, fake_target)   # l1(sig_B, fake_B)
                test_real_fake_lossA = l1(source, fake_source)

                #  ------------------------------- #
                #  ----- test discriminators ----- #
                #  ------------------------------- #

                test_d_loss, test_disc_A_loss, test_disc_B_loss = get_disc_loss(source, target, phase, intervention,
                                                                    disc_A, disc_B, gen_A, gen_B, 
                                                                    fake_source, fake_target
                                                                )
                
                # -------------------------------- #
                # ------- test generators -------- #
                # -------------------------------- # 

                out = calc_gen_loss(source, target, phase, intervention,
                                        gen_A, gen_B, disc_A, disc_B, 
                                        fake_target, fake_source
                                        )
                g_lossT, g_A_lossT, g_B_lossT, cycle_B_lossT, cycle_A_lossT, id_B_lossT, id_A_lossT, sup_A_lossT, sup_B_lossT = out
        
                # gen_signals(fake_target, fake_source, target, source)

            wandb.log({'Test/Generator loss': g_lossT.item(),
                        'Test/Discriminator loss': test_d_loss.item(),
                        'Test/L1 loss between real signal A and fake signals A': test_real_fake_lossA.item(),
                        'Test/L1 loss between real signal B and fake signals B': test_real_fake_lossB.item(),
                        'Test/Discriminator A loss': test_disc_A_loss.item(),
                        'Test/Discriminator B loss': test_disc_B_loss.item(),
                        'Test/Adversarial or GAN loss A': g_A_lossT.item(),
                        'Test/Adversarial or GAN loss B': g_B_lossT.item(),
                        'Test/Cycle consistency loss A': cycle_A_lossT.item(),
                        'Test/Cycle consistency loss B': cycle_B_lossT.item(),
                        'Test/Supervised loss A': sup_A_lossT.item(),
                        'Test/Supervised loss B': sup_B_lossT.item(),
                        'Test/Epoch': epoch+1,
                })
            
            
      
    disc_scheduler.step(test_d_loss)
    gen_scheduler.step(g_lossT)

    

In [ ]:
# Generate signals
idx = 0             
phases = df['Phasenzuordnung'].unique()
for source, target, phase, intervention in gen_loader:
    if idx == 3:
        break                 
                    
    source = source.float()
    target = target.float()
    source = source.to(DEVICE)
    target = target.to(DEVICE)
    phase = phase.to(DEVICE)
    intervention = intervention.to(DEVICE)

    fake_target = gen_B(source, phase, intervention)
    fake_source = gen_A(target, phase, intervention)

                        
    fake_target = fake_target.reshape(-1)
    fake_source = fake_source.reshape(-1)
    source = source.reshape(-1)
    target = target.reshape(-1)

    fig, ax = plt.subplots(2, 1, figsize=(10, 8))
    ax[0].plot(source.cpu().detach().numpy(), label= 'Real source signals')
    ax[0].plot(fake_source.cpu().detach().numpy(), label= 'Recreated source signals')
    ax[0].set_xlabel('Signal length')
    ax[0].set_ylabel('Loss')
    ax[0].legend()

    ax[1].plot(target.cpu().detach().numpy(), label= 'Real target signal')
    ax[1].plot(fake_target.cpu().detach().numpy(), label= 'Recreated target signal')
    ax[1].set_xlabel('Signal length')
    ax[1].set_ylabel('Loss')
    ax[1].legend()

    idx += 1       

In [ ]:
source_list = []
target_list = []

for source, target, phase, intervention in test_loader:
    # convert to float16
    source = source.float()
    target = target.float()

    # move to GPU
    source = source.to(DEVICE)
    target = target.to(DEVICE)
    phase = phase.to(DEVICE)
    intervention = intervention.to(DEVICE)

    fake_target = gen_B(source, phase, intervention).detach() # already torch.no_grad()
    fake_source = gen_A(target, phase, intervention).detach()

    # calculate l1 loss of fake signals and real signals
    target_loss= mse(target, fake_target)   # l1(sig_B, fake_B)
    source_loss = l1(source, fake_source)

    source_list.append(source_loss.item())
    target_list.append(target_loss.item())

    

print(len(source_list))
print(len(target_list))

In [ ]:
sum(source_list)/len(source_list)